### In this nootbook I will plot the performance of the 16-64 layer model (minimodel)

In [11]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from minimodel import data, metrics, model_builder, model_trainer

data_path = '../data'
weight_path = './checkpoints_16-64'
pretrained_weight_path = './checkpoints_16-320'
results_path = './results_16-64'
device = torch.device('cuda')

In [12]:
mouse_id = 5
file_name = "results_" + str(mouse_id) + ".npz"
results_file_path = os.path.join(results_path, file_name)
data_mini = np.load(results_file_path)

FEV_scores_mini = data_mini["FEV_scores"].squeeze()
FEVE_scores_mini = data_mini["FEVE_scores"].squeeze()
neurons_index_mini = data_mini["neurons_index"].squeeze()

In [13]:
print(f"FEV: {FEV_scores_mini}")
print(f"FEVE: {FEVE_scores_mini}")
print(f"index: {neurons_index_mini}")

FEV: [0.3172226  0.22012351]
FEVE: [0.6173496  0.68837357]
index: [438 244]


In [14]:
# ------ Data Loading ------

# load images
img = data.load_images(data_path, mouse_id, file=data.img_file_name[mouse_id])
nimg, Ly, Lx = img.shape
print('img: ', img.shape, img.min(), img.max(), img.dtype)

# load neurons
fname = '%s_nat60k_%s.npz'%(data.db[mouse_id]['mname'], data.db[mouse_id]['datexp'])
spks, istim_train, istim_test, xpos, ypos, spks_rep_all = data.load_neurons(file_path = os.path.join(data_path, fname), mouse_id = mouse_id)
n_stim, n_max_neurons = spks.shape

# split train and validation set
itrain, ival = data.split_train_val(istim_train, train_frac=0.9)

# normalize spks
spks, spks_rep_all = data.normalize_spks(spks, spks_rep_all, itrain)

ineur = np.arange(0, n_max_neurons) #np.arange(0, n_neurons, 5)

spks_val = torch.from_numpy(spks[ival][:,ineur]) 
spks_rep_all = [spks_rep_all[i][:,ineur] for i in range(len(spks_rep_all))]

img_val = torch.from_numpy(img[istim_train][ival]).to(device).unsqueeze(1)
img_test = torch.from_numpy(img[istim_test]).to(device).unsqueeze(1)

input_Ly, input_Lx = img_test.shape[-2:]

raw image shape:  (68000, 66, 264)
cropped image shape:  (68000, 66, 130)
img:  (68000, 66, 130) -2.0829253 2.1060908 float32
img:  (68000, 66, 130) -2.0829253 2.1060908 float32

loading activities from ../data/FX20_nat60k_2023_09_29.npz

splitting training and validation set...
itrain:  (43081,)
ival:  (4787,)

normalizing neural data...
finished
